# AutoUSD, sUSDe, Aave USDC and BIL (1-3 month T-Bill ETF) Comparison

Task:


> 
> Metrics:
> APY  
> Yield Volatility (annualized)  
> Excess vs 3M T-Bill  
> Information Ratio  
> For products:  
> USD: autoUSD, Ethena sUSDe, Aave USDC lending, US Gov short term treasuries  
> ETH: autoETH, Lido staking (stETH), Aave ETH lending  
> Time periods: 30 day, 1 year, since inception  
> *no 1 year on autoUSD since <1 year old  
> Definitions:  
> Yield Volatility: stdev of monthly net returns × √12  
> Excess vs Benchmark: difference between average net APY and the benchmark (3M T-bill for USD; Lido staking rate for ETH).  
> Information Ratio: average monthly excess return ÷ stdev of monthly excess returns, annualized  


# Methods


## Data sources:

Onchain calls on mainnet at the last block of each day

- autoUSD.converToAssets(1e18) / 1e6 -> USDC value of one autoUSD share
- sUSDe -> chainlink sUSDE -> USD oracle, safe USD value of sUSDe over time.
- Aave USDC lending, stataAaveUSDC.convertToAssets(1e6) / 1e6, growth of a share of the aave USDC vault

BIL `SPDR Bloomberg 1-3 Month T-Bill ETF`

https://www.ssga.com/us/en/intermediary/etfs/spdr-bloomberg-1-3-month-t-bill-etf-bil 

> The SPDR® Bloomberg  1-3 Month T-Bill ETF seeks to provide investment results that, before fees and expenses, correspond generally to the price and yield performance of the Bloomberg  1-3 Month U.S. Treasury Bill Index (the "Index")  
> Seeks to provide exposure to publicly issued U.S. Treasury Bills that have a remaining maturities between 1 and 3 months  
> Short duration fixed income is less exposed to fluctuations in interest rates than longer duration securities  
> Rebalanced on the last business day of the month  


As of Oct 26, 2025 BIL has ~42.5B AUM

```
!pip install yfinance
import yfinance as yf
tickers = ['BIL']
data = yf.download(tickers, start="2024-10-26", end="2025-10-26", actions=True)
data.to_csv('data.csv')

```



In [1]:
from multicall import Call

import pandas as pd
import numpy as np
import plotly.express as px

from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
)
from mainnet_launch.constants import ETH_CHAIN, AUTO_ETH


# https://etherscan.io/address/0x73edDFa87C71ADdC275c2b9890f5c3a8480bC9E6#readProxyContract
# chainlink safe price of sUSDe over time
# https://etherscan.io/address/0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c aave USDC
# https://data.chain.link/feeds/ethereum/mainnet/susde-usd


def _process_chainlink_usd_price(success, args):
    """
    Normalize Chainlink latestRoundData -> price as float (answer / 1e6).
    args is: (roundId:uint80, answer:int256, startedAt:uint256, updatedAt:uint256, answeredInRound:uint80)
    """
    round_id, answer, started_at, updated_at, answered_in_round = args
    if success:
        return float(answer) / 1e18


stETH_ETH_chainlink_oracle = "0x86392dC19c0b719886221c78AB11eb8Cf5c52812"

chainlink_stETH_price_call = Call(
    stETH_ETH_chainlink_oracle,
    ["latestRoundData()((uint80,int256,uint256,uint256,uint80))"],
    [("stETH_ETH_safe_price", _process_chainlink_usd_price)],
)

stata_aave = "0xC077F28FC0f10C886253E67Be402A7F7D611D4EE"  # convert to shares calls


aave_convert_to_shares_call = Call(
    stata_aave,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [("WETH_aave", safe_normalize_with_bool_success)],
)

autoETH_nav_per_share_call = Call(
    AUTO_ETH.autopool_eth_addr,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [("autoETH", safe_normalize_with_bool_success)],
)

wstETH = "0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0"

wstETH_convert_to_shares_call = Call(
    wstETH,
    ["stEthPerToken()(uint256)"],
    [("wstETH", safe_normalize_with_bool_success)],
)


insta_dapp_stETH_looping = "0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78"

ieth_convert_to_assets_call = Call(
    insta_dapp_stETH_looping,
    ["convertToAssets(uint256)(uint256)", int(1e18)],  # 1.0 share → assets
    [("iETH_v2", safe_normalize_with_bool_success)],
)


end_of_day_oct_26_2024_block = 21053236
blocks = build_blocks_to_use(ETH_CHAIN, start_block=end_of_day_oct_26_2024_block)
df = get_raw_state_by_blocks(
    calls=[
        aave_convert_to_shares_call,
        chainlink_stETH_price_call,
        autoETH_nav_per_share_call,
        wstETH_convert_to_shares_call,
        ieth_convert_to_assets_call,
    ],
    blocks=blocks,
    chain=ETH_CHAIN,
)

df["wstETH_eth_price"] = df["wstETH"] * df["stETH_ETH_safe_price"]
# df["iETH_v2_eth_price"] = df["iETH_v2"] * df["stETH_ETH_safe_price"]
df = df[["autoETH", "WETH_aave", "wstETH_eth_price"]]
end_of_day_prices = df[(df.index >= "2024-11-01") & (df.index <= "2025-11-01")].copy()

/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/eth_abi/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
from mainnet_launch.adhoc.returns_math import (
    compute_series_apy,
    compute_most_recent_30_days_apy,
    compute_yield_volatility_annualized,
    compute_excess_vs_benchmark_apy,
    compute_information_ratio,
    compute_30_day_excess_vs_benchmark_apy,
)

start_date = end_of_day_prices.index.min()
end_date = end_of_day_prices.index.max()
thrity_days_before_end = end_date - pd.Timedelta(days=30)
all_time_apy = compute_series_apy(end_of_day_prices)
most_recent_30_days_apy = compute_most_recent_30_days_apy(end_of_day_prices)
yield_volatility = compute_yield_volatility_annualized(end_of_day_prices)
excess_apy_vs_benchmark = compute_excess_vs_benchmark_apy(end_of_day_prices, "wstETH_eth_price")
most_recent_excess_vs_benchmark = compute_30_day_excess_vs_benchmark_apy(end_of_day_prices, "wstETH_eth_price")
information_ratio = compute_information_ratio(end_of_day_prices, "wstETH_eth_price")

combined_df = pd.concat(
    [
        all_time_apy.rename("All Time APY"),
        most_recent_30_days_apy.rename("30 Day APY"),
        yield_volatility.rename("Yield Volatility"),
        excess_apy_vs_benchmark.rename("Excess APY vs Benchmark"),
        most_recent_excess_vs_benchmark.rename("30 Day Excess vs Benchmark"),
        information_ratio.rename("Information Ratio"),
    ],
    axis=1,
)
print(f"Data from {start_date.date()} to {end_date.date()}")
print(combined_df.round(2).to_markdown())

Data from 2024-11-01 to 2025-10-31
|                  |   All Time APY |   30 Day APY |   Yield Volatility |   Excess APY vs Benchmark |   30 Day Excess vs Benchmark |   Information Ratio |
|:-----------------|---------------:|-------------:|-------------------:|--------------------------:|-----------------------------:|--------------------:|
| autoETH          |           5.19 |         1.27 |               0.65 |                      2.23 |                        -1.9  |                0.96 |
| WETH_aave        |           2.05 |         1.78 |               0.08 |                     -0.91 |                        -1.39 |               -0.65 |
| wstETH_eth_price |           2.96 |         3.17 |               0.3  |                      0    |                         0    |              nan    |
